In [1]:
from py5paisa import FivePaisaClient
cred={
    "APP_NAME":"5P59695694",
    "APP_SOURCE":"5367",
    "USER_ID":"ulGCwQinUw6",
    "PASSWORD":"ggniVTHwhMA",
    "USER_KEY":"M0Y3qsUJXwqOoJGxA5nGw5OfyKxGoxg8",
    "ENCRYPTION_KEY":"3iRMtiLFk7LrgToWCUwyMMETjte3LAoM"
    }

In [2]:
%%writefile keys.conf
[KEYS]
APP_NAME=5P59695694
APP_SOURCE=5367
USER_ID=ulGCwQinUw6
PASSWORD=ggniVTHwhMA
USER_KEY=M0Y3qsUJXwqOoJGxA5nGw5OfyKxGoxg8
ENCRYPTION_KEY=3iRMtiLFk7LrgToWCUwyMMETjte3LAoM


Overwriting keys.conf


In [3]:

client = FivePaisaClient(email="jawaadfazal@gmail.com", passwd="Siddiq@123", dob="19910309",cred=cred)
client.login()

 11:03:51 | Logged in!!


In [4]:
import pandas as pd
from pymongo import MongoClient
import pymongo
import json
import warnings
warnings.filterwarnings('ignore')


In [5]:
df=pd.read_csv(r'scripmaster-csv-format.csv')


In [6]:
historic_df=client.historical_data('N','C',3045,'1m','2021-05-25','2021-06-16')
print(historic_df)

                 Datetime    Open    High     Low   Close   Volume
0     2021-05-25T09:15:00  415.20  415.20  413.20  413.95  1646734
1     2021-05-25T09:16:00  414.10  414.65  413.95  414.40   875024
2     2021-05-25T09:17:00  414.45  414.75  413.80  413.80   551491
3     2021-05-25T09:18:00  413.65  414.00  413.40  413.95   367896
4     2021-05-25T09:19:00  413.90  414.00  413.10  413.15   464096
...                   ...     ...     ...     ...     ...      ...
6386  2021-06-16T15:26:00  425.70  425.80  425.50  425.70    61724
6387  2021-06-16T15:27:00  425.65  425.80  425.35  425.55    47505
6388  2021-06-16T15:28:00  425.55  425.80  425.50  425.70    43052
6389  2021-06-16T15:29:00  425.70  425.70  425.15  425.60    55102
6390  2021-06-16T15:30:00  425.60  425.60  425.60  425.60      329

[6391 rows x 6 columns]


In [34]:
class hist_data:
    #historic_df=pd.DataFrame()
    def __init__(self,exch,exty,name,timef,year):
        self.name=name
        cashdf=df[(df.Exch==exch)& (df.ExchType==exty)]
        cashdf['Name']=cashdf['Name'].str.strip()
        scrip=cashdf[cashdf['Name']==name]['Scripcode'].iat[0]
        historic_df1=client.historical_data(exch,exty,scrip,timef,f'{year}-01-01',f'{year}-06-30')
        historic_df2=client.historical_data(exch,exty,scrip,timef,f'{year}-07-01',f'{year}-12-31')
        #print(historic_df1)
        self.historic_df=historic_df1.append(historic_df2,ignore_index = True)
     
    def hist_show(self):
        print(self.historic_df)

class hist_data_csv(hist_data):
    
    def data_to_csv(self):
        self.historic_df.to_csv(f"{self.name}_csv.csv")
    
class import_mongo(hist_data_csv):
    db_url='mongodb://localhost:27017'
    db_name='data_base'
    
    def mongoimport(self):
        #def mongoimport(self,db_name, coll_name, db_url):
        self.data_to_csv()
        self.csv_path=f"{self.name}_csv.csv"
        self.coll_name=f"{self.name}_coll"
        
        client = MongoClient(self.db_url)
        db = client[self.db_name]  
        coll = db[self.coll_name]
        data = pd.read_csv(self.csv_path)
        df_=pd.DataFrame(data)
        posts = df_.to_dict(orient="record")
        #print(posts)
        coll.insert_many(posts)
        coll.create_index([("_id", pymongo.ASCENDING)])
    
    def insert_one(self,dict):
        client = MongoClient(self.db_url)
        db = client[self.db_name]  
        self.coll = db[self.coll_name]
        self.coll.insert_one(dict)
        
    def read(self,dict):
        client = MongoClient(self.db_url)
        db = client[self.db_name]  
        self.coll = db[self.coll_name]
        print(self.coll.find_one(dict))
        
    def update_one(self,dict1,data2):
        client = MongoClient(self.db_url)
        db = client[self.db_name]  
        self.coll = db[self.coll_name]
        self.coll.update_one(dict1,data2)
    
    def delete_one(self,dict):
        client = MongoClient(self.db_url)
        db = client[self.db_name]  
        self.coll = db[self.coll_name]
        self.coll.delete_one(dict)
        
        
               
#obj_SBIN=hist_data('N','C','SBIN','1d','2020')
#obj_RELIANCE=hist_data('N','C','RELIANCE','1d','2020')
#obj_TCS=hist_data('N','C','TCS','1d','2020')
#obj_LT=hist_data('N','C','LT','1d','2020')
#obj_IOC=hist_data('N','C','IOC','1d','2020')

#obj_SBIN=hist_data_csv('N','C','SBIN','1d','2020')
#obj_RELIANCE=hist_data_csv('N','C','RELIANCE','1d','2020')
#obj_TCS=hist_data_csv('N','C','TCS','1d','2020')
#obj_LT=hist_data_csv('N','C','LT','1d','2020')
#obj_IOC=hist_data_csv('N','C','IOC','1d','2020')

#obj_SBIN.data_to_csv()
#obj_RELIANCE.data_to_csv()
#obj_TCS.data_to_csv()
#obj_LT.data_to_csv()
#obj_IOC.data_to_csv()

obj_SBIN=import_mongo('N','C','SBIN','1d','2020')
obj_RELIANCE=import_mongo('N','C','RELIANCE','1d','2020')
obj_TCS=import_mongo('N','C','TCS','1d','2020')
obj_LT=import_mongo('N','C','LT','1d','2020')
obj_IOC=import_mongo('N','C','IOC','1d','2020')

obj_SBIN.mongoimport()
di={"Open":334.7}
obj_SBIN.read(di)

#obj1.hist_show()

    

{'_id': ObjectId('631dd2a45c3d0a3e14837c22'), 'Unnamed: 0': 0, 'Datetime': '2020-01-01T00:00:00', 'Open': 334.7, 'High': 335.95, 'Low': 332.15, 'Close': 334.45, 'Volume': 17379320}


In [58]:
class live_data_to_mongo:
    db_url='mongodb://localhost:27017'
    db_name='live_data_base'
    post={}
    def __init__(self,exch,exty,name):
        self.exch=exch
        self.exty=exty
        self.name=name
        cashdf=df[(df.Exch==exch)& (df.ExchType==exty)]
        cashdf['Name']=cashdf['Name'].str.strip()
        self.scrip=cashdf[cashdf['Name']==name]['Scripcode'].iat[0]

    def live_mongo(self):
        req_list=[
            { "Exch":self.exch,"ExchType":self.exty,"ScripCode":int(self.scrip)},
            ]
        req_data=client.Request_Feed('mf','s',req_list)
        
        def on_message(ws, message):
            print(message)
            t=message.strip("[")
            self.post=json.loads(t.strip("]"))
            coll.insert_one(self.post)
            
        self.coll_name=f"{self.name}_coll"    
        client_mdb = MongoClient(self.db_url)
        db = client_mdb[self.db_name]  
        coll = db[self.coll_name]
        client.connect(req_data)

        client.receive_data(on_message)

        
        

           #data = pd.read_csv(csv_path)
           #df_=pd.DataFrame(data)
            #posts = df_.to_dict(orient="record")
        #print(type(post))
        
        #coll.create_index([("_id", pymongo.ASCENDING)])

In [60]:
obj34=live_data_to_mongo('N','C','RELIANCE')
obj34.live_mongo()

 11:48:44 | Streaming Started
[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600.5,"LastQty":22,"TotalQty":1504933,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2589.95,"Time":20923,"BidQty":42,"BidRate":2600.5,"OffQty":199,"OffRate":2600.95,"TBidQ":199036,"TOffQ":927781,"TickDt":"\/Date(1662963523000)\/","ChgPcnt":1.21433854}]
[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600.95,"LastQty":83,"TotalQty":1505016,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2589.95,"Time":20925,"BidQty":42,"BidRate":2600.5,"OffQty":116,"OffRate":2600.95,"TBidQ":199030,"TOffQ":927800,"TickDt":"\/Date(1662963525000)\/","ChgPcnt":1.231853}]
[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600.9,"LastQty":1,"TotalQty":1505017,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2589.95,"Time":20925,"BidQty":42,"BidRate":2600.5,"OffQty":383,"OffRate":2600.9,"TBidQ":198780,"TOffQ":928308,"TickDt":"\/Date(1662963525000)\

[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600.45,"LastQty":1481,"TotalQty":1518609,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2590.04,"Time":20950,"BidQty":13,"BidRate":2600.05,"OffQty":100,"OffRate":2600.45,"TBidQ":196191,"TOffQ":928163,"TickDt":"\/Date(1662963550000)\/","ChgPcnt":1.21239245}]
[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600.05,"LastQty":123,"TotalQty":1518732,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2590.04,"Time":20951,"BidQty":112,"BidRate":2600.05,"OffQty":77,"OffRate":2600.45,"TBidQ":196545,"TOffQ":928158,"TickDt":"\/Date(1662963551000)\/","ChgPcnt":1.19682407}]
[{"Exch":"N","ExchType":"C","Token":2885,"LastRate":2600,"LastQty":560,"TotalQty":1519292,"High":2602.4,"Low":2570.55,"OpenRate":2570.55,"PClose":2569.3,"AvgRate":2590.05,"Time":20953,"BidQty":522,"BidRate":2600,"OffQty":77,"OffRate":2600.45,"TBidQ":195758,"TOffQ":928107,"TickDt":"\/Date(1662963553000)\/","ChgPcnt":1.194878}]